In [1]:
n = 30

weights = {
    1: 2,   2: 5,   3: 3,   4: 23,  5: 17,
    6: 8,   7: 12,  8: 1,   9: 4,  10: 5,
    11: 7,  12: 2,  13: 4,  14: 7,  15: 20,
    16: 5,  17: 22, 18: 7,  19: 4,  20: 1,
    21: 14, 22: 0,  23: 26, 24: 16, 25: 6,
    26: 15, 27: 8,  28: 4,  29: 8,  30: 4
}

durations = {
    1: 18,  2: 71,  3: 1,   4: 4,   5: 30,
    6: 97,  7: 73,  8: 83,  9: 34, 10: 22,
    11: 33, 12: 30, 13: 35, 14: 63, 15: 25,
    16: 84, 17: 87, 18: 83, 19: 46, 20: 63,
    21: 76, 22: 60, 23: 8,  24: 85, 25: 85,
    26: 26, 27: 83, 28: 86, 29: 36, 30: 82
}


ready_dates = {
    1: 4,     2: 28,    3: 85,    4: 69,    5: 373,
    6: 397,   7: 413,   8: 425,   9: 410,   10: 478,
    11: 696,  12: 768,  13: 812,  14: 902,  15: 903,
    16: 906,  17: 1012, 18: 1113, 19: 1163, 20: 1079,
    21: 1335, 22: 1294, 23: 1330, 24: 1497, 25: 1432,
    26: 1587, 27: 1338, 28: 1341, 29: 929,  30: 982
}


due_dates = {
    1: 83,    2: 619,   3: 649,   4: 724,   5: 1117,
    6: 565,   7: 1237,  8: 593,   9: 1001, 10: 677,
    11: 1370, 12: 1167, 13: 1348, 14: 1175, 15: 1000,
    16: 1212, 17: 1238, 18: 1990, 19: 1908, 20: 1323,
    21: 1915, 22: 2143, 23: 2037, 24: 2135, 25: 1981,
    26: 1870, 27: 2029, 28: 1641, 29: 1437, 30: 1084
}

deadlines = {
    1: 2201,  2: 1270,  3: 1213,  4: 2356,  5: 2392,
    6: 636,   7: 1773,  8: 1362,  9: 3116, 10: 2786,
    11: 2037, 12: 2393, 13: 3582, 14: 1627, 15: 2062,
    16: 3511, 17: 3599, 18: 4220, 19: 2048, 20: 2703,
    21: 2856, 22: 3939, 23: 3812, 24: 3881, 25: 3363,
    26: 3804, 27: 2312, 28: 3626, 29: 3426, 30: 2384
}


successors = {
    1: [2],
    2: [8, 7, 3],
    3: [11, 6],
    4: [8, 6],
    5: [6],
    6: [19, 12, 9],
    7: [19, 12, 9],
    8: [19, 9],
    9: [23, 17, 15, 14, 13],
    10: [22, 19, 17, 14, 13],
    11: [23, 19, 17, 13],
    12: [23, 16, 13],
    13: [21, 20, 18],
    14: [21, 20, 18],
    15: [22, 20, 18],
    16: [22, 21, 18],
    17: [21, 18],
    18: [28, 25, 24],
    19: [24, 21],
    20: [25, 24],
    21: [27, 25],
    22: [30, 24],
    23: [30, 24],
    24: [27, 26],
    25: [30, 26],
    26: [29],
    27: [29],
    28: [29],
    29: [],
    30: []
}



In [2]:
predecessors = {i: [] for i in range(1, n+1)}
for i in range(1, n+1):
    for s in successors[i]:
        predecessors[s].append(i)


In [3]:
from collections import deque

indeg = {i: len(predecessors[i]) for i in range(1, n+1)}
q = deque([i for i in range(1, n+1) if indeg[i] == 0])
topo = []

while q:
    u = q.popleft()
    topo.append(u)
    for v in successors[u]:
        indeg[v] -= 1
        if indeg[v] == 0:
            q.append(v)

if len(topo) != n:
    raise ValueError("Precedence graph has a cycle!")


In [4]:
new_ready_dates = {i: ready_dates[i] for i in range(1, n+1)}
for i in topo:
    if predecessors[i]:
        new_ready_dates[i] = max(ready_dates[i], max(new_ready_dates[j] + durations[j] for j in predecessors[i]))

In [5]:
new_deadlines = {i: deadlines[i] for i in range(1, n+1)}

for i in reversed(topo):
    if successors[i]:
        new_deadlines[i] = min(new_deadlines[i], min(new_deadlines[s] - durations[s] for s in successors[i]))

In [8]:
from pysat.formula import CNF, IDPool
from pysat.solvers import Glucose42
from pysat.card import CardEnc, EncType


def solve_with_prefix_precedence(n, durations, ready_dates, deadlines, successors):
    jobs = list(range(1, n+1))
    t_max = max(deadlines.values())

    vpool = IDPool()
    cnf = CNF()

    # =========================================================
    # 1. VARIABLE CREATION
    # =========================================================
    S = {}      # S[i,t] = job i starts at time t
    A = {}      # A[i,t] = job i active at time t
    SC = {}     # SC[j,t] = job j start time ≤ t  (prefix variable)
    valid = {}  # valid start times

    for i in jobs:
        r = ready_dates[i]
        p = durations[i]
        d = deadlines[i]

        last_start = d - p
        if last_start < r:
            print(f"Job {i} impossible: ready>{last_start}")
            return False, None

        valid[i] = list(range(r, last_start+1))

        # Create S vars
        for t in valid[i]:
            S[(i,t)] = vpool.id(("S", i, t))

        # Create A vars
        for t in range(r, d):
            A[(i,t)] = vpool.id(("A", i, t))


    # =========================================================
    # 2. EXACTLY-ONE START FOR EACH JOB
    # =========================================================
    for i in jobs:
        enc = CardEnc.equals(
            lits=[S[(i,t)] for t in valid[i]],
            bound=1,
            encoding=EncType.seqcounter
        )
        cnf.extend(enc.clauses)


    # =========================================================
    # 3. S → A (activation)
    # =========================================================
    for i in jobs:
        p = durations[i]
        for t0 in valid[i]:
            s_lit = S[(i,t0)]
            for t in range(t0, t0 + p):
                if (i,t) in A:
                    cnf.append([-s_lit, A[(i,t)]])


    # =========================================================
    # 4. CAPACITY: AT MOST 1 JOB ACTIVE AT TIME t
    # =========================================================
    for t in range(t_max):
        lits = [A[(i,t)] for i in jobs if (i,t) in A]
        if len(lits) > 1:
            enc = CardEnc.atmost(lits=lits, bound=1, encoding=EncType.seqcounter)
            cnf.extend(enc.clauses)


    # =========================================================
    # 5. PREFIX VARIABLES SC[j,t]
    #    SC[j,t] = 1 iff ∃u ≤ t: S[j,u] = 1
    # =========================================================
    for j in jobs:
        times = valid[j]
        if not times:
            continue
        t_min = times[0]
        t_maxj = times[-1]

        # Create SC[j,t]
        for t in range(t_min, t_maxj+1):
            SC[(j,t)] = vpool.id(("SC", j, t))

        # (A) S[j,t] → SC[j,t]
        for t in times:
            cnf.append([-S[(j,t)], SC[(j,t)]])

        # (B) SC[j,t] → SC[j,t+1]
        for t in range(t_min, t_maxj):
            cnf.append([-SC[(j,t)], SC[(j,t+1)]])

        # (C) SC[j,t] → (S[j,u1] ∨ S[j,u2] ... u ≤ t)
        for t in range(t_min, t_maxj+1):
            ors = [S[(j,u)] for u in times if u <= t]
            cnf.append([-SC[(j,t)]] + ors)

        # Optional: last SC must be true (job must start ≤ max)
        #cnf.append([SC[(j,t_maxj)]])


    # =========================================================
    # 6. PRECEDENCE ENCODING: i → j
    #    S[i,t_i] → ¬SC[j, finish_i]
    # =========================================================
    for i in jobs:
        for j in successors.get(i, []):
            p_i = durations[i]
            if not valid[j]:
                continue
            t_min_j, t_max_j = valid[j][0], valid[j][-1]

            for t_i in valid[i]:
                finish = t_i + p_i - 1

                # SC only defined within [t_min_j, t_max_j]
                if finish < t_min_j or finish > t_max_j:
                    continue

                cnf.append([-S[(i,t_i)], -SC[(j, finish)]])


    # =========================================================
    # 7. SOLVE
    # =========================================================
    solver = Glucose42()
    solver.append_formula(cnf)

    if not solver.solve():
        print("UNSAT — no feasible schedule.")
        return False, None

    model = set(solver.get_model())

    # Extract schedule
    schedule = {}
    for (i,t), vid in S.items():
        if vid in model:
            schedule[i] = t

    # Output sorted by start time
    print("\n===== FEASIBLE SCHEDULE =====\n")
    print("{:<8} {:<10} {:<10}".format("Job", "Start", "End"))
    print("-"*30)

    order = sorted(schedule.items(), key=lambda x: x[1])
    for i, start in order:
        end = start + durations[i]
        print("{:<8} {:<10} {:<10}".format(i, start, end))

    print("\nORDER:", [i for i,_ in order])

    return True, schedule


In [9]:
solve_with_prefix_precedence(n, durations, new_ready_dates, new_deadlines, successors)

UNSAT — no feasible schedule.


(False, None)